In [1]:
from subprocess import Popen, call, check_call, STDOUT, PIPE
import os
from tempfile import NamedTemporaryFile
import pandas as pd
from numpy import mean
from tqdm import tqdm
from collections import defaultdict
from multiprocessing import Pool, Process
from time import sleep
import pandas as pd

In [2]:
os.chdir("/home/urban/mhc/")

In [3]:
MHCI = "/home/urban/mhc1/mhc_i/src/predict_binding.py"
#methods = ["ann", "pickpocket", "smm", "netmhcpan"]
methods = ["pickpocket", "smm", "netmhcpan"]
#methods = ["netmhccons"]

In [4]:
def run_mhci(sequence, method, allele, length, resmin=True):
    tf = NamedTemporaryFile(suffix=".fa", delete=False)
    tf.writelines([">1\n", sequence])
    tf.close()
    proc = Popen([MHCI, method, allele, str(length), tf.name], stdin=PIPE, stdout=PIPE, stderr=PIPE)
    proc.wait()
    os.unlink(tf.name)
    result = proc.communicate()[0]
    try:
        target_index = result.split('\n')[0].split('\t').index("ic50")
    except ValueError:
        return "NA"
    targets = [float(l.split('\t')[target_index]) for l in result.split('\n')[1:] if l]
    if targets:
        return min(targets) if resmin else targets
    else:
        return "NA"

In [5]:
def run_mhci_meth(sequence, methods, allele, length, aggregate = mean):
    return aggregate([run_mhci(sequence, meth, allele, length) for meth in methods])

In [6]:
def run_mhci_allen(sequence, method, pairs):
    return {sequence:[(p[0]+"/"+p[1], run_mhci(sequence, method, p[0], p[1]) ) for p in pairs]}


In [7]:
def run_mhci_allen_(par):
    sequence = par['sequence']
    method = par['method']
    pairs = par['pairs']
    return (sequence, [(p[0]+"/"+p[1], run_mhci(sequence, method, p[0], p[1]) ) for p in pairs])


In [8]:
def run_mhci_peptides(peptides, alleles, method, processes=22):
    result = []
    pool = Pool(processes=processes)
    args = [{'sequence':s, 'method':method, 'pairs':alleles} for s in peptides]
    for res in tqdm(pool.imap_unordered(run_mhci_allen_, args), total=len(peptides)):
        result.append(res)
    pool.close()
    pool.join()
    return result

In [9]:
def write_res_table(results, filename):
    with open(filename, "w") as f:
        f.write("peptide\tmodel\tic50\n")
        for pep, allist in results:
            for al, ic50 in allist:
                f.write("{}\t{}\t{}\n".format(pep, al, ic50))

In [10]:
allelesi = [(l.split(',')[0].strip(), l.split(',')[1].strip() ) for l in open("hla_ref_set.class_i.txt").readlines()]
randHS = [l.strip() for l in open("4368_HS_random_peptides.txt").readlines()]
randMB = [l.strip() for l in open("4368_MB_random_peptides.txt").readlines()]
randNC = [l.strip() for l in open("4368_NC_random_peptides.txt").readlines()]

In [11]:
ind = pd.read_table("hIndividuals.tsv")

ind.set_index("sequence", inplace=True)

individuals = {}
for c in ind.columns:
    individuals[c] = list(ind[ind.loc[:,c]>0].index.values)
    with open(c+"_peps.txt", "w") as f:
        for p in individuals[c]:
            f.write(p+'\n')

In [ ]:
os.chdir("/home/urban/mhc/")
for m in methods:
    outdir = os.path.join("h1", m)
    os.mkdir(outdir)
    for name, peps in zip(["randHS", "randMB", "randNC"],[randHS, randMB, randNC]):
        print "Running {}:{}".format(m,name)
        r = run_mhci_peptides(peps, allelesi, m)
        write_res_table(r, os.path.join(outdir, name+".tsv"))
    
    for name, peps in individuals.items():
        print "Running {}:{}".format(m,name)
        r = run_mhci_peptides(peps, allelesi, m)
        write_res_table(r, os.path.join(outdir, name+".tsv"))
    

Running pickpocket:randHS


100%|██████████| 4368/4368 [13:59:06<00:00,  5.63s/it]    /4368 [03:27<251:35:35, 207.40s/it]  1%|          | 22/4368 [03:50<12:37:50, 10.46s/it]

Running pickpocket:randMB



 44%|████▍     | 1915/4368 [6:03:19<30:14:08, 44.37s/it]